In [2]:
import numpy as np
import pandas as pd


In [5]:
# mydata= pd.read_csv("C:\\Users\\Deepanshu\\Documents\\file1.csv")

url = 'https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/train.csv'
train_df = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/remcovanwijk040/AI/main/kaggle/titanic_competition/test.csv'
test_df = pd.read_csv(url)

print("train set:", train_df.shape)
print("test set:", test_df.shape)

train set: (891, 12)
test set: (418, 11)


## Check the data

In [8]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Check the numeric values:

In [10]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Check categorical values:

In [41]:
list = train_df[['Sex', 'Ticket', 'Embarked', 'Cabin']]

for column in list.columns:
     print("\n" + column)
     print(train_df[column].value_counts())


Sex
male      577
female    314
Name: Sex, dtype: int64

Ticket
347082      7
CA. 2343    7
1601        7
3101295     6
CA 2144     6
           ..
9234        1
19988       1
2693        1
PC 17612    1
370376      1
Name: Ticket, Length: 681, dtype: int64

Embarked
S    644
C    168
Q     77
Name: Embarked, dtype: int64

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64


## Fix missings

We will impute the missing outcome values with the most occurring value, and use the preprocessing functionality of sklearn for this task. We will use this functionality again after splitting the data to impute missings in the predictors, as this will ensure that we do the exact same preprocessing on the predictors in the test set.


from sklearn.impute import SimpleImputer
imp_outcome = SimpleImputer(strategy = 'most_frequent')
imp_outcome.fit(df['Class'].to_frame())

## Split the training data

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(df, df['Class']):
  strat_train_set = df.loc[train_index]
  strat_test_set = df.loc[test_index]

In [ ]:
#check if this resulted in a 70% train set and 30% test set with equal outcome distributions
print('share of observations in the train set:', str(round(len(strat_train_set)/len(df), 3)))
print('share of observations in the test set:', str(round(len(strat_test_set)/len(df), 3)))
print('share of mines in the train set:', str(round(len(strat_train_set.loc[strat_train_set['Class'] == 'Mine'])/len(strat_train_set), 3)))
print('share of mines in the test set:', str(round(len(strat_test_set.loc[strat_test_set['Class'] == 'Mine'])/len(strat_test_set), 3)))


In [ ]:
# split features and outcome data
X_train = strat_train_set.loc[:,strat_train_set.columns != 'Class']
y_train = strat_train_set['Class']
X_test = strat_test_set.loc[:,strat_test_set.columns != 'Class']
y_test = strat_test_set['Class']

## Setting up the pipeline

Setting up the pipeline
Machine learning is about trial and error, with typically many iterations between different ways to prepare the data (data preparation) and assessing how that translates into performance (model building).

Going through different iterations should require the least possible effort, which is done by setting up a pipeline, using sklearn's pipeline functionality. A pipeline explicitly operationalizes all the different steps taken to get from raw data into data prepared and ready to serve as modeling input.

As such, the pipeline serves two purposes:

it encompasses every preprocessing step, coded within an overarching Python object, such that it is easily transferred to new data (the raw test data or tomorrow's raw process data). This approach prevents 'manual' adjustments to the data, that would get lost or need to be repeated 'manually' when preparing new data.
it makes it really easy to tweak the preprocessing steps, trying out different ways to prepare the data, requiring minimal adjustment to your code (less time-consuming and less error-prone)
Typically, the preprocessing steps include:

selecting the variables
imputing missing values
scaling numeric variables
creating dummy variables for categorical variables
The first thing to realize is that numeric and categorical features require different preprocessing steps, effectively meaning that our overarching pipeline should contain two parallel pipelines: one for preparing numeric features and one for preparing categorical features